In [4]:
import os
import cv2
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from skimage.metrics import peak_signal_noise_ratio, structural_similarity

SOTS_IN_HAZY = "../data/RESIDE/SOTS/indoor/hazy"
SOTS_IN_GT   = "../data/RESIDE/SOTS/indoor/gt"
RESULT_DCP_IN = "../results/dcp/indoor"

os.makedirs(RESULT_DCP_IN, exist_ok=True)


In [5]:
hazy_files = sorted([f for f in os.listdir(SOTS_IN_HAZY) if f.endswith("_10.png")])
len(hazy_files)


50

In [7]:
def dark_channel(im, size=15):
    """
    im: float32, [0,1], shape (H,W,3)
    """
    b, g, r = cv2.split(im)
    min_img = cv2.min(cv2.min(r, g), b)
    # 最小滤波（形态学腐蚀）
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (size, size))
    dark = cv2.erode(min_img, kernel)
    return dark


def estimate_atmospheric_light(im, dark, top_percent=0.001):
    """
    估计大气光 A
    im: float32, [0,1], (H,W,3)
    dark: float32, [0,1], (H,W)
    """
    h, w = dark.shape
    num_pixels = max(int(h * w * top_percent), 1)
    dark_vec = dark.reshape(-1)
    im_vec = im.reshape(-1, 3)

    # 取暗通道中最亮的若干像素索引
    indices = np.argsort(dark_vec)[-num_pixels:]
    A = im_vec[indices].mean(axis=0)  # 对这些位置的RGB取平均
    return A


def estimate_transmission(im, A, omega=0.95, size=15):
    """
    估计透射率 t(x)
    """
    norm_im = im / A  # 逐通道归一化
    t = 1 - omega * dark_channel(norm_im, size)
    return t


def recover_image(im, t, A, t0=0.1):
    """
    恢复无雾图像 J(x)
    """
    t = np.clip(t, t0, 1)
    J = (im - A) / t[:, :, None] + A
    J = np.clip(J, 0, 1)
    return J


def dehaze_dcp(image_path, size=15, omega=0.95, t0=0.1):
    """
    用 DCP 对单张图去雾，返回 uint8 RGB 图
    """
    # 读图 & 归一化
    im = cv2.imread(image_path)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB).astype(np.float32) / 255.0

    dark = dark_channel(im, size=size)
    A = estimate_atmospheric_light(im, dark)
    t = estimate_transmission(im, A, omega=omega, size=size)
    
    # 简单平滑一下 t，减弱块状感（可视化更好）
    t = cv2.blur(t, (15, 15))
    
    J = recover_image(im, t, A, t0=t0)
    return (J * 255).astype(np.uint8)


In [8]:
for fname in tqdm(hazy_files, desc="Indoor dehazing"):
    hazy_path = os.path.join(SOTS_IN_HAZY, fname)

    # extract GT name
    base = fname.split("_")[0]  # "1400"
    gt_name = base + ".png"
    
    out_path = os.path.join(RESULT_DCP_IN, fname)

    # use best parameters from tuning
    J = dehaze_dcp(hazy_path, size=31, omega=0.88, t0=0.15)

    cv2.imwrite(out_path, cv2.cvtColor(J, cv2.COLOR_RGB2BGR))


Indoor dehazing:   0%|          | 0/50 [00:00<?, ?it/s]

In [9]:
rows = []

for fname in tqdm(hazy_files, desc="Compute indoor metrics"):
    hazy_path = os.path.join(SOTS_IN_HAZY, fname)
    base = fname.split("_")[0]
    gt_path = os.path.join(SOTS_IN_GT, base + ".png")
    dcp_path = os.path.join(RESULT_DCP_IN, fname)

    if not os.path.exists(gt_path):
        continue

    gt  = cv2.cvtColor(cv2.imread(gt_path), cv2.COLOR_BGR2RGB)
    dcp = cv2.cvtColor(cv2.imread(dcp_path), cv2.COLOR_BGR2RGB)

    if gt.shape != dcp.shape:
        dcp = cv2.resize(dcp, (gt.shape[1], gt.shape[0]))

    psnr = peak_signal_noise_ratio(gt, dcp, data_range=255)
    ssim = structural_similarity(gt, dcp, data_range=255, channel_axis=2)

    rows.append((fname, base + ".png", psnr, ssim))

df_in = pd.DataFrame(rows, columns=["hazy_file", "gt_file", "psnr", "ssim"])
df_in.to_csv("../metrics/dcp_sots_indoor_psnr_ssim_tuned.csv", index=False)

df_in.head(), df_in[["psnr","ssim"]].describe()


Compute indoor metrics:   0%|          | 0/50 [00:00<?, ?it/s]

(     hazy_file   gt_file       psnr      ssim
 0  1400_10.png  1400.png  16.341846  0.806612
 1  1401_10.png  1401.png  19.455092  0.817630
 2  1402_10.png  1402.png  19.837597  0.763029
 3  1403_10.png  1403.png  15.437532  0.713375
 4  1404_10.png  1404.png  21.256637  0.900921,
             psnr       ssim
 count  50.000000  50.000000
 mean   18.184093   0.810212
 std     2.095590   0.074738
 min    11.921486   0.579784
 25%    16.649813   0.767699
 50%    18.413870   0.820273
 75%    19.810030   0.862859
 max    21.605076   0.906313)